<a href="https://colab.research.google.com/github/tam1444AH/COSC4397Project/blob/main/notebooks/supervised-data-preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install -U "huggingface-hub<1.0" accelerate peft bitsandbytes
!pip -q install -U ms-swift transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 46.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.6/449.6 kB 25.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.8/882.8 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 133.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.6/564.6 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.6/416.6 kB 35.7 MB/s eta 

In [2]:
import shutil, transformers, huggingface_hub, sys, os
os.environ["PATH"] += ":/root/.local/bin:/usr/local/bin"
print("swift bin:", shutil.which("swift"))
print("ms-swift bin:", shutil.which("ms-swift"))
print("transformers:", transformers.__version__)
print("huggingface_hub:", huggingface_hub.__version__)

swift bin: /usr/local/bin/swift
ms-swift bin: None
transformers: 4.57.1
huggingface_hub: 0.36.0


In [3]:
!pip -q install -U hf_transfer
!export HF_HUB_ENABLE_HF_TRANSFER=1

import os
from google.colab import userdata
from huggingface_hub import login, whoami
import wandb

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"  # mitigate fragmentation
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
HF_TOKEN = userdata.get('HF_TOKEN')
WANDB_TOKEN = userdata.get('WANDB_KEY')
os.environ["WANDB_API_KEY"] = WANDB_TOKEN
os.environ["HF_TOKEN"] = HF_TOKEN
wandb.login(key=WANDB_TOKEN, relogin=True)
login(token=HF_TOKEN, add_to_git_credential=True)  # also sets Git creds for LFS

print("Logged in as:", whoami(token=HF_TOKEN)["name"])

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mohammedtalha290 (mohammedtalha290-university-of-houston) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Logged in as: tam2003


In [5]:
from pathlib import Path
import json, random
from collections import defaultdict
random.seed(42)

dataset_path = Path("/content/test.jsonl")
rows = [json.loads(line) for line in dataset_path.read_text(encoding="utf-8").splitlines() if line.strip()]

supervised_rows = [row for row in rows if row.get("set") == "supervised"]
print(f"Total rows: {len(rows)}, Supervised rows: {len(supervised_rows)}")

if len(supervised_rows) == 0:
    print("WARNING: No supervised examples found! Check your 'set' field.")
    supervised_rows = rows

by_filetype = defaultdict(list)
for row in rows:
    filetype = row.get("filetype", "unknown")
    by_filetype[filetype].append(row)

train, val = [], []
for filetype, examples in by_filetype.items():
    random.shuffle(examples)
    cut = max(1, int(0.95 * len(examples)))
    train.extend(examples[:cut])
    val.extend(examples[cut:])
    print(f"{filetype}: {len(examples)} total → {len(examples[:cut])} train, {len(examples[cut:])} val")

random.shuffle(train)
random.shuffle(val)

train_path = Path("/content/train.jsonl")
val_path = Path("/content/val.jsonl")
train_path.write_text("\n".join(json.dumps(row, ensure_ascii=False) for row in train), encoding="utf-8")
val_path.write_text("\n".join(json.dumps(row, ensure_ascii=False) for row in val), encoding="utf-8")

print(f"\nFinal split - Train: {len(train)}, Val: {len(val)}")
print(f"Saved to {train_path} and {val_path}")

Total rows: 75, Supervised rows: 18
cryptol: 75 total → 71 train, 4 val

Final split - Train: 71, Val: 4
Saved to /content/train.jsonl and /content/val.jsonl


In [ ]:
%%bash

export SWIFT_USE_HF=1

swift sft \
  --use_hf true \
  --model Qwen/Qwen2.5-Coder-32B \
  --adapters tam2003/peft-FT-2.5-Coder-32b \
  --dataset /content/train.jsonl \
  --val_dataset /content/val.jsonl \
  --learning_rate 1e-4 \
  --num_train_epochs 1 \
  --warmup_ratio 0.03 \
  --per_device_train_batch_size 2 \
  --gradient_accumulation_steps 8 \
  --torch_dtype float16 \
  --gradient_checkpointing true \
  --lora_r 16 \
  --lora_alpha 32 \
  --lora_dropout 0.05 \
  --save_steps 200 \
  --logging_steps 20 \
  --output_dir /content/sft_output

In [ ]:
!pip -q install -U trl datasets

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from trl import SFTTrainer
from datasets import load_dataset
from peft import LoraConfig, TaskType

In [ ]:
model_name = "Qwen/Qwen3-32B-Coder-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

train_dataset = load_dataset("json", data_files="/content/train.jsonl", split="train")
val_dataset = load_dataset("json", data_files="/content/val.jsonl", split="train")

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

training_args = TrainingArguments(
    output_dir="/content/sft_output",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=1e-4,
    warmup_ratio=0.03,
    logging_steps=20,
    save_steps=200,
    eval_steps=200,
    evaluation_strategy="steps",
    fp16=True,
    gradient_checkpointing=True,
    report_to="wandb",
    push_to_hub=True,
    hub_model_id="your-username/qwen3-coder-cryptol-sft",
    hub_token=HF_TOKEN,
)

# Train
trainer.train()

# Save and push
trainer.save_model("/content/sft_output/final")
trainer.push_to_hub()
